In [1]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

# Generate creds for the Drive FUSE library.
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

E: Package 'python-software-properties' has no installation candidate
Selecting previously unselected package google-drive-ocamlfuse.
(Reading database ... 131323 files and directories currently installed.)
Preparing to unpack .../google-drive-ocamlfuse_0.7.1-0ubuntu3~ubuntu18.04.1_amd64.deb ...
Unpacking google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Setting up google-drive-ocamlfuse (0.7.1-0ubuntu3~ubuntu18.04.1) ...
Processing triggers for man-db (2.8.3-2ubuntu0.1) ...
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=ht

In [0]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

In [143]:
import os
os.listdir("drive/Colab/data")

['customShapeNet', 'ShapeNetRendering', 'latentVectors', 'output_clouds']

In [8]:
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'
accelerator


'cu80'

In [9]:
#!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.0-{platform}-linux_x86_64.whl torchvision
!pip install torch
import torch
torch.__version__

'1.0.1.post2'

In [0]:
import os
import time

import numpy as np

from torch import nn

In [0]:
from PIL import Image

def loadImage(path):
    # Now that we have an img, we need to preprocess it.
    img = Image.open(path)
    
    # We need to:
    #       * resize the img, it is pretty big (~1200x1200px).
    #       * normalize it, as noted in the PyTorch pretrained models doc,
    #         with, mean = [0.485, 0.456, 0.406] and std = [0.229, 0.224, 0.225].
    #       * convert it to a PyTorch Tensor.
    # We can do all this preprocessing using a transform pipeline.
    img_size = 224  # The min size, as noted in the PyTorch pretrained models doc, is 224 px.
    transform_pipeline = transforms.Compose([transforms.Resize(img_size),
                                             transforms.ToTensor(),
                                             transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                                                  std=[0.229, 0.224, 0.225])])
    
    img = transform_pipeline(img)
    
    # PyTorch pretrained models expect the Tensor dims to be (num input imgs, num color channels, height, width).
    # Currently however, we have (num color channels, height, width); let's fix this by inserting a new axis.
    #img = img.unsqueeze(0)  # Insert the new axis at index 0 i.e. in front of the other axes/dims.
    img = img[:3, :, :].unsqueeze(0)  #enlève le channel alpha
    
    # Now that we have preprocessed our img, we need to convert it into a
    # Variable; PyTorch models expect inputs to be Variables. A PyTorch Variable is a
    # wrapper around a PyTorch Tensor.
    return torch.tensor(img)


In [0]:
import sys
import numpy as np
import pandas as pd
from collections import defaultdict

ply_dtypes = dict([
    (b'int8', 'i1'),
    (b'char', 'i1'),
    (b'uint8', 'u1'),
    (b'uchar', 'b1'),
    (b'uchar', 'u1'),
    (b'int16', 'i2'),
    (b'short', 'i2'),
    (b'uint16', 'u2'),
    (b'ushort', 'u2'),
    (b'int32', 'i4'),
    (b'int', 'i4'),
    (b'uint32', 'u4'),
    (b'uint', 'u4'),
    (b'float32', 'f4'),
    (b'float', 'f4'),
    (b'float64', 'f8'),
    (b'double', 'f8')
])

valid_formats = {'ascii': '', 'binary_big_endian': '>',
                 'binary_little_endian': '<'}


def read_ply(filename):
    """ Read a .ply (binary or ascii) file and store the elements in pandas DataFrame
    Parameters
    ----------
    filename: str
        Path tho the filename
    Returns
    -------
    data: dict
        Elements as pandas DataFrames; comments and ob_info as list of string
    """

    with open(filename, 'rb') as ply:

        if b'ply' not in ply.readline():
            raise ValueError('The file does not start whith the word ply')
        # get binary_little/big or ascii
        fmt = ply.readline().split()[1].decode()
        # get extension for building the numpy dtypes
        ext = valid_formats[fmt]

        line = []
        dtypes = defaultdict(list)
        count = 2
        points_size = None
        mesh_size = None
        while b'end_header' not in line and line != b'':
            line = ply.readline()

            if b'element' in line:
                line = line.split()
                name = line[1].decode()
                size = int(line[2])
                if name == "vertex":
                    points_size = size
                elif name == "face":
                    mesh_size = size

            elif b'property' in line:
                line = line.split()
                # element mesh
                if b'list' in line:
                    mesh_names = ['n_points', 'v1', 'v2', 'v3']

                    if fmt == "ascii":
                        # the first number has different dtype than the list
                        dtypes[name].append(
                            (mesh_names[0], ply_dtypes[line[2]]))
                        # rest of the numbers have the same dtype
                        dt = ply_dtypes[line[3]]
                    else:
                        # the first number has different dtype than the list
                        dtypes[name].append(
                            (mesh_names[0], ext + ply_dtypes[line[2]]))
                        # rest of the numbers have the same dtype
                        dt = ext + ply_dtypes[line[3]]

                    for j in range(1, 4):
                        dtypes[name].append((mesh_names[j], dt))
                else:
                    if fmt == "ascii":
                        dtypes[name].append(
                            (line[2].decode(), ply_dtypes[line[1]]))
                    else:
                        dtypes[name].append(
                            (line[2].decode(), ext + ply_dtypes[line[1]]))
            count += 1

        # for bin
        end_header = ply.tell()

    data = {}

    if fmt == 'ascii':
        top = count
        bottom = 0 if mesh_size is None else mesh_size

        names = [x[0] for x in dtypes["vertex"]]

        data["points"] = pd.read_csv(filename, sep=" ", header=None, engine="python",
                                     skiprows=top, skipfooter=bottom, usecols=names, names=names)

        for n, col in enumerate(data["points"].columns):
            data["points"][col] = data["points"][col].astype(
                dtypes["vertex"][n][1])

        if mesh_size is not None:
            top = count + points_size

            names = [x[0] for x in dtypes["face"]][1:]
            usecols = [1, 2, 3]

            data["mesh"] = pd.read_csv(
                filename, sep=" ", header=None, engine="python", skiprows=top, usecols=usecols, names=names)

            for n, col in enumerate(data["mesh"].columns):
                data["mesh"][col] = data["mesh"][col].astype(
                    dtypes["face"][n + 1][1])

    else:
        with open(filename, 'rb') as ply:
            ply.seek(end_header)
            data["points"] = pd.DataFrame(np.fromfile(
                ply, dtype=dtypes["vertex"], count=points_size))
            if mesh_size is not None:
                data["mesh"] = pd.DataFrame(np.fromfile(
                    ply, dtype=dtypes["face"], count=mesh_size))
                data["mesh"].drop('n_points', axis=1, inplace=True)

    return data


def write_ply(filename, points=None, faces = None, mesh=None, as_text=False, normal=False, text=False, color = False):
    """
    Parameters
    ----------
    filename: str
        The created file will be named with this
    points: ndarray
    mesh: ndarray
    as_text: boolean
        Set the write mode of the file. Default: binary
    Returns
    -------
    boolean
        True if no problems
    """
    if not filename.endswith('ply'):
        filename += '.ply'

    # open in text mode to write the header
    with open(filename, 'w') as ply:
        header = ['ply']

        if as_text:
            header.append('format ascii 1.0')
        else:
            header.append('format binary_' + sys.byteorder + '_endian 1.0')

        if points is not None:
            header.extend(describe_element('vertex', points, normal, text, color))
        if mesh is not None:
            mesh = mesh.copy()
            mesh.insert(loc=0, column="n_points", value=3)
            mesh["n_points"] = mesh["n_points"].astype("u1")
            header.extend(describe_element('face', mesh, normal, text, color))
        if faces is not None:
            header.extend(describe_element('face', faces, text))
        header.append('end_header')

        for line in header:
            ply.write("%s\n" % line)

    if as_text:
        if points is not None:
            # print("write points")
            points.to_csv(filename, sep=" ", index=False, header=False, mode='a',
                          encoding='ascii')
            # print("end points")
        
        if faces is not None:
            faces.to_csv(filename, sep=" ", index=False, header=False, mode='a',
                          encoding='ascii')
        
        if mesh is not None:
            mesh.to_csv(filename, sep=" ", index=False, header=False, mode='a',
                        encoding='ascii')

    else:
        # open in binary/append to use tofile
        with open(filename, 'ab') as ply:
            if points is not None:
                points.to_records(index=False).tofile(ply)
            if faces is not None:
                faces.to_records(index=False).tofile(ply)
            if mesh is not None:
                mesh.to_records(index=False).tofile(ply)
        # Read in the file
    # with open(filename, 'r') as file :
    #   filedata = file.read()

    # # Replace the target string
    # filedata = filedata.replace('.', ',')
    # filedata = filedata.replace('ascii 1,0', 'ascii 1.0')

    # # Write the file out again
    # with open(filename, 'w') as file:
    #   file.write(filedata)
    return True


def describe_element(name, df, normal=False, text = False, color=False):
    """ Takes the columns of the dataframe and builds a ply-like description
    Parameters
    ----------
    name: str
    df: pandas DataFrame
    Returns
    -------
    element: list[str]
    """
    property_formats = {'f': 'float', 'u': 'uchar', 'i': 'int'}
    element = ['element ' + name + ' ' + str(len(df))]

    if name == 'face':
        element.append("property list uchar int vertex_indices")

    else:
        element.append('property float x')
        element.append('property float y')
        element.append('property float z')
        
        if text:
           element.append('property float u')
           element.append('property float v')
        
        if normal:
            element.append('property float nx')
            element.append('property float ny')
            element.append('property float nz')
        if color:
            element.append('property uchar red')
            element.append('property uchar green')
            element.append('property uchar blue')

    return element



def _gen_latent(root, id_category, nPerCat, nPerObj):
    """renvoie un vecteur d'images chargées dans l'ordre
    taille: id_category * nPerCat * nPerObj"""
    res = []
    for cat in id_category:
        path = root + str(cat) + "/"
        for key in os.listdir(path)[:nPerCat]:
            sub_path = path + key + "/rendering/"
            
            with open(sub_path + "renderings.txt") as f:
                cpt = 0
                for line in f:
                    res.append(loadImage(sub_path + line[:-1]))
                    cpt += 1
                    if cpt == nPerObj:
                        break
    
    return res  #torch.utils.data.DataLoader(res, nPerCat)


def _gen_clouds(root, id_category, nPerCat, ratio_sous_echantillonage):
    root += "/"
    res = []
    for cat in id_category:
        path = root + str(cat) + "/ply/"
        cpt = 0
        for key in os.listdir(path):
            #ignore les fichiers .txt
            if key[-4:] != ".txt":
                sub_path = path + key
                cloud = read_ply(sub_path)['points']
                
                sub_sampled = []
                for i, x in enumerate(cloud.values[:, :3]):
                    if len(sub_sampled) < ratio_sous_echantillonage * (i+1):
                        sub_sampled.append(torch.tensor(x))
                        
                res.append(sub_sampled)
                
                cpt += 1
                if cpt == nPerCat:
                    break
    
    return np.array(res, dtype=torch.Tensor)


def _save_latent(latent, root, id_category, nPerCat, nPerObj):
    cpt=0
    for cat in id_category:
        path = root + "/" + str(cat)
        if not os.path.isdir(path):
            os.mkdir(path)
        for ind_obj in range(nPerCat):
            path2 = path + "/" + str(ind_obj)
            if not os.path.isdir(path2):
                os.mkdir(path2)
            for ind_view in range(nPerObj):
                path3 = path2 + "/" + str(ind_view)
                if not os.path.isfile(path3):
                    np.save(path3, latent[cpt])
                cpt+=1


def _load_latent(root, id_category, nPerCat, nPerObj):
    latent = []
    for cat in id_category:
        path = root + "/" + str(cat)
        for ind_obj in range(nPerCat):
            path2 = path + "/" + str(ind_obj)
            for ind_view in range(nPerObj):
                path3 = path2 + "/" + str(ind_view) + ".npy"
                latent.append(torch.tensor(np.load(path3)))
    return latent


def _get_categories(path, chosenSubSet):
    path += "/synsetoffset2category.txt"
    
    id_category = []
    with open(path) as f:
        for line in f:
            id_category.append(line.split()[1])
    
    #récupère les catégories des indices demandés
    if chosenSubSet:
        id_category = [id_category[i] for i in chosenSubSet]
    return id_category


def get_latent(chosenSubSet, nPerCat, nPerObj):
    id_category = _get_categories("drive/Colab/data/ShapeNetRendering/", chosenSubSet)
    root = "../AtlasNet/data/ShapeNetRendering"
    local_path = "drive/Colab/data/latentVectors"
    
    #chargement des vecteurs latents
    latentVectors = _load_latent(local_path, id_category, nPerCat, nPerObj)
    
    return latentVectors


def get_clouds(chosenSubSet, nPerCat, ratio):
    assert 0 < ratio <= 1
    path = "drive/Colab/data/customShapeNet"
    id_category = _get_categories(path, chosenSubSet)
    return _gen_clouds(path, id_category, nPerCat, ratio)


def write_clouds(path, clouds):
    if os.path.isdir(path):
        print(path, "écrasé")
    else :
        print(path, "écrit")
        os.mkdir(path)
    path += '/'
    for i, c in enumerate(clouds):
        ply.write_ply(path + str(i), pd.DataFrame(c), as_text=True)

In [124]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
class Segmentation:
    """défini la manière de partitionner l'espace 3D en cases"""
    
    def __init__(self, n_step):
        assert n_step % 2 == 1, "n_step doit être impair pour que le centre soit une cellule"
        self.dim = 3
        self.coo_min = -1
        self.coo_max = 1
        self.n_step = n_step
        self.n_step_2 = n_step / 2
        
        self.neighbours, self.distances_2 = self._gen_neighbours()
        # for d in self.distances_2:
        #     print(d,":",self.neighbours[d])
        
        # sert à initialiser la matrice
        self.filler = np.frompyfunc(lambda x: list(), 1, 1)
    
    def _gen_neighbours(self):
        # génère tous les déplacements possibles de -n à n dans chaque dim
        r = range(self.n_step * 2)
        list_neighbours = np.array([[[(i, j, k) for i in r] for j in r] for k in r])
        
        # shift de n pour centrer
        list_neighbours = [voisin - self.n_step for voisin in list_neighbours.reshape(-1, 3)]
        
        # facteur pour passer de distances entre cases (coordonnées entières) à des distances entre points
        factor_2 = ((self.coo_max - self.coo_min) / self.n_step) ** 2
        
        # trie par distance au centre
        # il y a bcp d'équivalents
        # on regroupe les équivalents dans une même liste
        dict_neighbours = {}
        for voisin in list_neighbours:
            # la distance minimale entre des points des deux cases est égale à
            # la norme du vecteur entre les cases auquel on ENLEVE 1 DANS TOUTES LES DIMENSIONS NON NULLES
            # on laisse tout au carré
            d2 = np.sum(np.power(np.maximum(np.abs(voisin) - 1, 0), 2)) * factor_2
            
            # une dimension égale à 1 devient équivalente à une dimension nulle
            # on ajoute un terme négligeable pour mettre en premiers les cases les plus proches du centre
            d2 += (np.abs(voisin) == 1).sum() * 1e-6
            
            if d2 not in dict_neighbours:  # pas de pb d'arrrondi car on travaille sur des entiers, puis effectue les mêmes opérations
                dict_neighbours[d2] = []
            
            dict_neighbours[d2].append(voisin)
        
        dict_neighbours = {k: np.array(v) for (k, v) in dict_neighbours.items()}
        
        # calcule la liste triée des distances
        list_dist_2 = np.array(sorted(list(dict_neighbours.keys())))
        
        return dict_neighbours, list_dist_2
    
    def get_neighbours(self, cell, dist):
        """retourne la liste des cases à distance dist de cell"""
        # ajoute notre position pour calculer les coordonnées des voisins
        res = cell + self.neighbours[dist]
        
        # filtre les voisins qui sortent du tableau
        return [v for v in res if np.all((0 <= v) & (v < self.n_step))]
    
    def gen_matrix(self):
        """retourne un tableau 3D servant à stocker les points"""
        a = np.empty((self.n_step, self.n_step, self.n_step), dtype=list)
        return self.filler(a, a)
    
    def get_mat_coo(self, point):
        """retourne les coordonnées de la case du tableau contenant ce point"""
        res = [int((coo + 1) * self.n_step_2) for coo in point]
        #<=> int((coo - self.coo_min) / (self.coo_max - self.coo_min) * self.n_step)
        
        # si une coordonnée est maximale (càd 1), le point sort du tableau, on le met dans la case d'avant
        return [min(x, self.n_step - 1) for x in res]
    
    def get_float_coo(self, point):
        """retourne les coordonnées de notre point dans la base des cases du tableau"""
        return np.minimum((point + 1) * self.n_step_2, self.n_step - 1)


In [0]:
class Nuage:
    """stocke l'ensemble des points sous la forme
    d'une liste de points et
    d'un tableau à 3 dimensions"""
    
    def __init__(self, points, segmentation):
        self.segmentation = segmentation
        self.liste_points = points
        
        # crée le tableau
        self.mat = segmentation.gen_matrix()
        
        # remplit le tableau
        for p in self.liste_points:
            x, y, z = self.segmentation.get_mat_coo(p)
            self.mat[x, y, z].append(p)
        
    def recreate(self, points):
        """vide et re-rempli l'ancien tableau plutôt que de le supprimer et d'en allouer un nouveau"""
        self.liste_points = points
        
        # vide
        self.segmentation.filler(self.mat, self.mat)
        
        # remplit
        for p in self.liste_points:
            x, y, z = self.segmentation.get_mat_coo(p)
            self.mat[x, y, z].append(p)
        
        return self
    

    # pour afficher des statistiques: liste du nombre de points parcourus pour chaque appel à get_closest
    points_parcourus = []
    
    def get_closest(self, point):
        # calcule la case dans laquelle tomberait le point
        case_coo = self.segmentation.get_mat_coo(point)
        
        # le point le plus proche ne se trouve pas forcément dans la case la plus proche
        # un point plus proche peut encore être trouvé dans une case plus lointaine tant que
        # la distance à l'angle le plus proche de cette case est inférieur au min actuel
        point_coo = self.segmentation.get_float_coo(point.to("cpu").detach().numpy())
        
        cpt_point_parcourus = 0
        closest_point = torch.tensor(float("inf")).to(device)
        for d_2 in self.segmentation.distances_2:
            # stoppe la recherche si la distance minimale entre les angles des deux cases n'est pas meilleure
            if d_2 >= closest_point:
                break
            
            # considère toutes les cases non vides à distance d de notre case
            neighbours = [v for v in self.segmentation.get_neighbours(case_coo, d_2) if self.mat[v[0], v[1], v[2]]]
            
            # calcule la distance entre notre point et le sommet de la case le plus proche
            precise_dist_2 = [(np.sum(np.power(point_coo - v, 2)), v) for v in neighbours]
            
            # enlève les cases dont la distance min est trop grande
            precise_dist_2 = [(d_2, v) for (d_2, v) in precise_dist_2 if d_2 < closest_point]
            
            # parcourt les cases par ordre croissant (pour réduire l'espérance de temps pour trouver le point le plus proche)
            for real_d_2, v in sorted(precise_dist_2, key=lambda x: x[0]):
                if real_d_2 > closest_point:
                    break
                    
                cpt_point_parcourus += len(self.mat[v[0], v[1], v[2]])
                
                # regarde si le point le plus proche parmi tous les points de la case est mieux
                candidat = torch.min(torch.cat([torch.sum(torch.pow(point - p, 2), dim=(0,)).unsqueeze(0) for p in self.mat[v[0], v[1], v[2]]]))
                if candidat < closest_point:
                    closest_point = candidat
            else:
                continue  # only executed if the inner loop did NOT break
            break  # only executed if the inner loop DID break
        
        Nuage.points_parcourus.append(cpt_point_parcourus)
        # return torch.sqrt(closest_point)
        return closest_point
    
    def chamfer(self, other):
        loss0 = torch.sum(torch.cat([self.get_closest(p).unsqueeze(0) for p in other.liste_points]))
        loss1 = torch.sum(torch.cat([other.get_closest(p).unsqueeze(0) for p in self.liste_points]))
        return loss0, loss1


In [0]:
class Reconstructeur(nn.Module):
    def __init__(self, n_mlp, latent_size, n_grid_step, segmentation):
        super().__init__()
        print("nombre de MLP:", n_mlp)
        print("résolution de la grille:", n_grid_step, "^2 =", n_grid_step ** 2)
        print("taille des nuages générés:", n_mlp, "*", n_grid_step ** 2, "=", n_mlp * n_grid_step ** 2)
        print("résolution segmentation:", segmentation.n_step, "^3 =", segmentation.n_step ** 3)
        self._nuage_tmp = Nuage([], segmentation)
        self.input_dim = 2
        self.output_dim = 3
        self.n_grid_step = n_grid_step
        self.verbose = True
        
        self.decodeurs = nn.ModuleList([nn.Sequential(
            #on rajoute les coordonnées d'un point dans l'espace de départ
            nn.Linear(latent_size + self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            #on retourne un point dans l'espace de sortie
            nn.Linear(128, self.output_dim),
            #tanh ramène dans [-1, 1], comme les nuages de notre auto-encodeur et de groueix
            nn.Tanh()
        ) for _ in range(n_mlp)])
        
        self.grid = self._gen_grid()
        self.loss = Nuage.chamfer
    
    def _gen_grid(self):
        """retourne une liste de n points 2D au bon format :
        array(tensor(coo_x, coo_y))
        les points forment un quadrillage du carré unitaire (il n'y a pas d'aléa)
        les tensors ont requires_grad=False car cela reviendrait à déformer la grille"""
        points_dim = [np.linspace(-1, 1, self.n_grid_step) for _ in range(self.input_dim)]
        points_dim = np.meshgrid(*points_dim)
        
        def f(*point_dim):
            v = torch.tensor(list(point_dim)).float()
            v.requires_grad = False
            return v.to(device)
        
        res = np.vectorize(f, otypes=[object])(*points_dim).reshape(-1)
        return res
    
    def forward(self, x):
        # concatenation de x et p pour tous les points de la grille
        grid_x = [torch.cat((x, p)) for p in self.grid]
        
        # f(x_p) pour tout x_p pour tout mlp
        return self._nuage_tmp.recreate(np.array([[f(x_p) for x_p in grid_x] for f in self.decodeurs]).reshape(-1))
    

In [0]:
def fit_reconstructeur(reconstructeur, x_train, y_train, x_test, y_test, epochs, lr=1e-5, grid_scale=1.0):
    """grid_scale:
        les points 3D générés sont calculés à partir d'un échantillonage d'un carré 1*1,
        prendre un carré 100*100 revient à augmenter les coefficients du
        premier Linear sans pénaliser le modèle (Adam y est en fait insensible par défaut)"""
    reconstructeur.grid *= grid_scale
    optimizer = torch.optim.Adam(reconstructeur.parameters(), lr=lr)
    loss_train = None
    
    time_loss = np.zeros(epochs)
    time_tot = np.zeros(epochs)
    
    for epoch in range(epochs):
        loss_train = 0
        t_tot = time.time()
        for x, y in zip(x_train, y_train):
            assert not x.requires_grad
            
            y_pred = reconstructeur.forward(x)
            
            t_loss = time.time()
            loss = reconstructeur.loss(y_pred, y)
            time_loss[epoch] += time.time() - t_loss
            
            # print(loss)
            loss = loss[0] + loss[1]
            loss_train += loss.item() / len(x_train)
            
            # Zero gradients, perform a backward pass, and update the weights.
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        time_tot[epoch] += time.time() - t_tot
        
        if reconstructeur.verbose and (epochs < 10 or epoch % (epochs // 10) == 0):
            reconstructeur.eval()
            
            s_test = sum(sum(reconstructeur.loss(reconstructeur.forward(x), y.data)).item() for (x, y) in zip(x_test, y_test))
            
            print("time", epoch,
                  "loss train %.4e" % loss_train,
                  "loss test %.2e" % s_test)
            reconstructeur.train()
        
        if loss_train / len(x_train) < 1e-8:
            #convergence finie, ca ne sert à rien de continuer
            break
    
    #apprentissage fini, passage en mode évaluation
    reconstructeur.eval()
    return loss_train, time_loss, time_tot

In [0]:
class Reconstructeur2(nn.Module):
    """chamfer quadratique, à supprimer"""
    def __init__(self, n_mlp, latent_size, n_grid_step):
        super().__init__()
        print("nombre de MLP:", n_mlp)
        print("résolution de la grille:", n_grid_step, "*", n_grid_step, "=", n_grid_step ** 2)
        print("taille des nuages générés:", n_mlp, "*", n_grid_step ** 2, "=", n_mlp * n_grid_step ** 2, )
        self.input_dim = 2
        self.output_dim = 3
        self.n_grid_step = n_grid_step
        self.verbose = True
        
        self.decodeurs = nn.ModuleList([nn.Sequential(
            #on rajoute les coordonnées d'un point dans l'espace de départ
            nn.Linear(latent_size + self.input_dim, 512),
            nn.ReLU(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Linear(256, 128),
            nn.ReLU(),
            #on retourne un point dans l'espace de sortie
            nn.Linear(128, self.output_dim),
            #tanh ramène dans [-1, 1], comme les nuages de notre auto-encodeur et de groueix
            nn.Tanh()
        ) for _ in range(n_mlp)])
        
        self.decodeurs = nn.ModuleList([nn.Sequential(
            nn.Linear(latent_size + self.input_dim, self.output_dim),
            nn.Tanh()
        ) for _ in range(n_mlp)])
        
        self.grid = self._gen_grid()
        self.loss = self.chamfer
    
    def _gen_grid(self):
        points_dim = [np.linspace(0, 1, self.n_grid_step) for _ in range(self.input_dim)]
        points_dim = np.meshgrid(*points_dim)
        
        def f(*point_dim):
            v = torch.tensor(list(point_dim)).float()
            v.requires_grad = False
            return v
        
        return np.vectorize(f, otypes=[object])(*points_dim).reshape(-1)
    
    @staticmethod
    def chamfer(Y, S):
        """return chamfer loss between
        the generated set Y = {f(x, p) for each f, p}
        and the real pointcloud S corresponding to the latent vector x

        sum_F(sum_A) et min_A(min_F) correspondent à une simple somme ou min sur l'ensemble des données
        donc on représente l'ensemble des points générés Y comme une liste et non comme une matrice
        """
    
        # Pas la même taille :
        # normes = torch.pow(torch.norm(Y - S), 2)
    
        # normes = np.array([[torch.norm(y - s) for s in S] for y in Y])
        normes = np.array([[torch.sum(torch.pow(y - s, 2), dim=(0,)) for s in S] for y in Y])
        loss1 = np.sum(np.min(normes, axis=0))
        loss2 = np.sum(np.min(normes, axis=1))
        return loss1, loss2
    
        # On sert uniquement des min selon les deux axes, on peut ne pas stocker la matrice pour éviter les problèmes de RAM
        # listes des minimums sur chaque ligne et colonne
        min_axe0 = [torch.tensor(float("+inf"))] * len(S)
        min_axe1 = [torch.tensor(float("+inf"))] * len(Y)
    
        # pour chaque case de la matrice, met à jour les deux minimums correspondants
        for i, s in enumerate(S):
            for j, y in enumerate(Y):
                val = torch.norm(y - s)
                min_axe0[i] = torch.min(val, min_axe0[i])
                min_axe1[j] = torch.min(val, min_axe1[j])
    
        # les Q doivent bien être atteint par un MLP
        loss1 = sum(min_axe0)
    
        # les MLP doivent bien atteindre un Q
        loss2 = sum(min_axe1)
    
        return loss1, loss2

    def forward(self, x):
        # concatenation de x et p pour tous les points de la grille
        grid_x = [torch.cat((x, p)) for p in self.grid]
        
        # f(x_p) pour tout x_p pour tout mlp
        return np.array([[f(x_p) for x_p in grid_x] for f in self.decodeurs]).reshape(-1)


In [145]:
chosen_subset = [0]
n_per_cat = 1
clouds2 = get_clouds(chosen_subset, n_per_cat, ratio=.01)
clouds2 = [np.array([x.to(device) for x in c]) for c in clouds2]
print("taille des nuages ground truth:", len(clouds2[0]))
latent = get_latent(chosen_subset, n_per_cat,
                         nPerObj=1)  # /!\ attention: il faut que les fichiers sur le disque correspondent
latent = [x.to(device) for x in latent]

segmentation = Segmentation(5)


clouds = [Nuage(x, segmentation) for x in clouds2]

# plot_tailles(clouds)
#write_clouds("./data/output_clouds/ground_truth", [[p.detach().numpy() for p in clouds[0]]])

ratio_train = 1
n_train = int(ratio_train * len(latent))

indexes = list(range(len(latent)))
#random.shuffle(indexes)
train_x = [latent[i] for i in indexes[:n_train]]
test_x = [latent[i] for i in indexes[n_train:]]
train_y = [clouds[i] for i in indexes[:n_train]]
test_y = [clouds[i] for i in indexes[n_train:]]
train_y2 = [clouds2[i] for i in indexes[:n_train]]
test_y2 = [clouds2[i] for i in indexes[n_train:]]

n_mlp = 10
latent_size = 25088  # défini par l'encodeur utilisé
grid_points = 8
epochs = 5
grid_size = 1e0
lr = 1e-4

reconstructeur = Reconstructeur(n_mlp, latent_size, grid_points, segmentation)
reconstructeur.to(device)

t = time.time()
for _ in range(1):
    loss, t_loss, t_tot = fit_reconstructeur(reconstructeur, train_x, train_y, test_x, test_y, epochs, lr=lr, grid_scale=grid_size)
    print("\ntemps: ")
    print("loss", sum(t_loss), t_loss)
    print("tot", sum(t_tot), t_tot)
    print("ratio", sum(t_loss)/sum(t_tot), t_loss/t_tot)
print("O(n)", time.time() - t)

print("répartition point parcourus")
print(np.histogram(Nuage.points_parcourus))
print(np.mean(Nuage.points_parcourus))
# print(segmentation.distances_2)

output = reconstructeur.forward(latent[0])
output = [p.cpu().detach().numpy() for p in output.liste_points]
write_clouds("drive/Colab/data/output_clouds", [output])

exit()
print("reconstructeur O(n2) : ")
reconstructeur2 = Reconstructeur2(n_mlp, latent_size, grid_points)
t = time.time()
for _ in range(1):
    fit_reconstructeur(reconstructeur2, train_x, train_y2, test_x, test_y2, epochs, lr=lr, grid_scale=grid_size)
print("O(n2)", time.time() - t)  #

taille des nuages ground truth: 300
nombre de MLP: 10
résolution de la grille: 8 ^2 = 64
taille des nuages générés: 10 * 64 = 640
résolution segmentation: 5 ^3 = 125
time 0 loss train 1.8595e+02 loss test 0.00e+00
time 1 loss train 1.1707e+02 loss test 0.00e+00
time 2 loss train 8.0666e+01 loss test 0.00e+00
time 3 loss train 6.6948e+01 loss test 0.00e+00
time 4 loss train 6.4388e+01 loss test 0.00e+00

temps: 
loss 31.65546178817749 [13.90408897  4.89564133  4.5984261   4.06980014  4.18750525]
tot 82.04366111755371 [40.42584419 11.48423839 10.94003129  9.55288982  9.64065742]
ratio 0.3858367771133585 [0.3439406  0.42629221 0.42033025 0.42602817 0.43435889]
O(n) 82.13453102111816
répartition point parcourus
(array([11373,   321,   249,   128,     4,     1,     0,     0,     0,
         600]), array([  4. ,  67.6, 131.2, 194.8, 258.4, 322. , 385.6, 449.2, 512.8,
       576.4, 640. ]))
66.97230987693278
drive/data/output_clouds écrit


FileNotFoundError: ignored

In [66]:
os.listdir("drive/Colab/data/latentVectors/02933112/0")

['0.npy']

cuda:0
